### Starting the function here

In [535]:
# Some test
import pandas as pd
import itertools 
from bs4 import BeautifulSoup
import requests
from requests import get
import time
from random import seed
from random import random
from random import randint
from ediblepickle import checkpoint
import os
import sys
import re
sys.setrecursionlimit(100000)


# specify the url format
first_page = 'https://streeteasy.com/for-rent/nyc'

cache_dir = 'cache_streeteasy'
if not os.path.exists(cache_dir):
    os.mkdir(cache_dir)

@checkpoint(key=lambda args, kwargs: args[0].replace('/','_') + '.pkl', work_dir=cache_dir)
def scrape_website(url):
    # initialize a list called houses 
    houses = []
    # initialize variable count at 1

    # initialize variable new_count at 0

    req_headers = {
        'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
        'accept-encoding': 'gzip, deflate, br',
        'accept-language': 'en-US,en;q=0.8',
        'upgrade-insecure-requests': '1',
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36'
    }

    # request the response
    with requests.Session() as s:
        response = s.get(url, headers=req_headers)

    # parse through the html 
    html_soup = BeautifulSoup(response.text, 'html.parser')
    # in the html of the page, find all the bins with <li> and class:
    #       house_data = html_soup.find_all('li', class_="search-list__item search-list__item--listing")
    house_data = html_soup.find_all('li', class_="searchCardList--listItem")
    # I like to print where the program is on the screen so we can follow its progress and where any errors happened
#    print(first_page)

    # if the response was not empty (if something was actually scraped)
    if house_data != []:
        # add to the list houses
        houses.extend(house_data)
        # random wait times
        value = random()
        scaled_value = 20 + (value * (3))
        print(scaled_value)
        time.sleep(scaled_value)

    return houses

In [400]:
# First Page
first_page = 'https://streeteasy.com/for-rent/nyc'

second_page = 'https://streeteasy.com/for-rent/nyc?page=31'

houses = scrape_website(second_page)

In [537]:
# Need to do a loop over all the pages

houses = []

# First webpage 
first_page = 'https://streeteasy.com/for-rent/nyc'

house_temp = scrape_website(first_page)

houses.extend(house_temp)

page_root = 'https://streeteasy.com/for-rent/nyc?page='

count = 31
while count < 33:
    url = page_root + str(count)
    print(url)
    house = scrape_website(url)
    houses.extend(house)
    count += 1


https://streeteasy.com/for-rent/nyc?page=31
https://streeteasy.com/for-rent/nyc?page=32


In [432]:
len(houses)

406

In [428]:
houses

[]

In [383]:
response

<Response [200]>

In [402]:
len(houses)

0

In [3]:
# Getting some properties from the website with the count

import re

house_price = []
house_location = []
house_address = []
house_feature = []
house_url = []
house_bedroom = []
house_bathroom = []
house_area = []

# Here is the for loop
count = 4

for count in range(len(houses)):
    # Price of the house
    num = houses[count]
    price = num.find_all('span',{"class":"price listingCard-priceMargin"})[0].text
    price = price.replace(',','').replace('$','')
    house_price.append(price)
    df_price = pd.DataFrame({'house_price':house_price})

    # Location
    # Check if there is a new building level
    temp = num.find_all('p',{"class":"listingCardLabel listingCardLabel-grey listingCard-upperShortLabel"})
    if len(temp) == 1:
        location = num.find_all('p',{"class":"listingCardLabel listingCardLabel-grey listingCard-upperShortLabel"})[0].text
    else:
        location = num.find_all('p',{"class":"listingCardLabel listingCardLabel-grey listingCard-upperShortLabel"})[1].text
    match_location = re.findall(r'(?<=in ).*', location.strip())
    house_location.append(str(match_location))
    df_location = pd.DataFrame({'location':house_location})
    
    # Address and url for the appartment
    if num.find_all('a',{"class":"listingCard-link jsCardLinkGA featured-link-to-hdp"}):
        address = num.find_all('a',{"class":"listingCard-link jsCardLinkGA featured-link-to-hdp"})[0].text
        url = num.find_all('a',{"class":"listingCard-link jsCardLinkGA featured-link-to-hdp"})[0]
        url = url.get('href')
    else:
        address = num.find_all('a',{"class":"listingCard-link jsCardLinkGA"})[0].text
        url = num.find_all('a',{"class":"listingCard-link jsCardLinkGA"})[0]
        url = url.get('href')
    house_address.append(address)
    df_address = pd.DataFrame({'address':house_address})
    house_url.append(url)
    df_url = pd.DataFrame({'url':house_url})
    
    
    # Number of bedrooms
    bedrooms = num.find_all('span',{"listingDetailDefinitionsText"})[0].text
    if bedrooms.strip() == 'Studio':
        house_bedroom.append(0)
    else:      
        house_bedroom.append(int(bedrooms.strip()[0]))
    df_bedrooms = pd.DataFrame({'bedroom':house_bedroom})
    
    
    # Number of bathroom    
    try:
        bath = num.find_all('span',{"class":"listingDetailDefinitionsText"})[1].text
        house_bathroom.append(int(bath.strip()[0]))
    except:
        house_bathroom.append(1)
        
    df_bathroom = pd.DataFrame({'bathroom':house_bathroom})
    
    # Surface area in sqf
    area = num.find_all('span',{"class":"listingDetailDefinitionsText"});
    if len(area)==3:
        try:
            house_area.append(int(area[2].get_text().strip().replace('\n','').split()[0]))
        except:
            house_area.append(int(area[2].get_text().strip().replace('\n','').split()[0].strip().replace(',','')))
    else:
        house_area.append(None)
    df_area = pd.DataFrame({'area':house_area})
    
    
#     <span class="listingDetailDefinitionsText">1 Bath</span>
    
#     <span class="listingDetailDefinitionsText">1 Bath</span>
    
    # # Here is the adress 
    # <address class="listingCard-addressLabel listingCard-upperShortLabel">
    #         <a class="listingCard-link jsCardLinkGA featured-link-to-hdp" tabindex="-1" href="https://streeteasy.com/building/31_36-32-street-astoria/4a?featured=1" data-featured-event-category="rentals_search" data-featured-event-label="listing_id=3873282|slot=1">31-36 32nd Street #4A</a>
    #       </address>

    # # Here is the neig
    # <p class="listingCardLabel listingCardLabel-grey listingCard-upperShortLabel">
    #         Rental Unit in Astoria
    #       </p>
    
#     <span class="listingDetailDefinitionsText">2 Beds</span>

result = pd.concat([df_price, df_location], axis=1, sort=False)
result1 = pd.concat([result, df_address], axis=1, sort=False)
result2 = pd.concat([result1, df_url], axis=1, sort=False)
result3 = pd.concat([result2,df_bedrooms], axis=1, sort=False)
result4 = pd.concat([result3,df_bathroom], axis=1, sort=False)
result5 = pd.concat([result4,df_area], axis=1, sort=False)

#dfa = pd.concat([result3, df_surface], axis=1, sort=False)
result5.head()

,house_price,location,address,url,bedroom,bathroom,area
0,4874,['Stuyvesant Town/PCV'],321 Avenue C #8H,https://streeteasy.com/building/315-avenue-c-n...,1,1,764.0
1,3300,['Sutton Place'],344 East 55th Street #2C,https://streeteasy.com/building/344-east-55-st...,1,1,NaN
2,2600,['Jersey City'],19 Cypress Street GROUND-F,https://streeteasy.com/building/19-cypress-str...,2,2,NaN
3,5195,['Murray Hill'],401 East 34th Street #N21G,https://streeteasy.com/building/view-34/n21g,1,1,636.0
4,3647,['Hunters Point'],28-10 Jackson Avenue #21K,https://streeteasy.com/building/28_10-jackson-...,1,1,NaN


In [729]:
len(result5)

15666

In [732]:
result5.url

0        https://streeteasy.com/building/315-avenue-c-n...
1        https://streeteasy.com/building/344-east-55-st...
2        https://streeteasy.com/building/19-cypress-str...
3             https://streeteasy.com/building/view-34/n21g
4        https://streeteasy.com/building/28_10-jackson-...
                               ...                        
15661    https://streeteasy.com/building/the-harper-at-...
15662    https://streeteasy.com/building/the-harper-at-...
15663    https://streeteasy.com/building/the-harper-at-...
15664    https://streeteasy.com/building/the-harper-at-...
15665    https://streeteasy.com/building/the-harper-at-...
Name: url, Length: 15666, dtype: object

In [279]:
# To test indiviudal list

house_price = []
house_location = []
house_address = []

# Here is the for loop
count = 2

# Price of the house
num = houses[count]
price = num.find_all('span',{"class":"price listingCard-priceMargin"})[0].text
price = price.replace(',','').replace('$','')
house_price.append(price)
df_price = pd.DataFrame({'house_price':house_price})

# Location
location = num.find_all('p',{"class":"listingCardLabel listingCardLabel-grey listingCard-upperShortLabel"})[0].text
match_location = re.findall(r'(?<=in ).*', location.strip())
house_location.append(str(match_location))
df_location = pd.DataFrame({'location':house_location})

# address
location = num.find_all('p',{"class":"listingCardLabel listingCardLabel-grey listingCard-upperShortLabel"})[0].text
match_location = re.findall(r'(?<=in ).*', location.strip())
house_location.append(str(match_location))
df_location = pd.DataFrame({'location':house_location})

bedrooms = num.find_all('span',{"listingDetailDefinitionsText"})[0].text

# <a class="listingCard-link jsCardLinkGA featured-link-to-hdp" tabindex="-1" href="https://streeteasy.com/building/31_36-32-street-astoria/4a?featured=1" data-featured-event-category="rentals_search" data-featured-event-label="listing_id=3873282|slot=1">31-36 32nd Street #4A</a>
# #result = pd.concat([df_price, df_location], axis=1, sort=False)


In [283]:
# url
url = num.find_all('span',{"class":"listingDetailDefinitionsText"})
# match_location = re.findall(r'(?<=in ).*', location.strip())
# house_location.append(str(match_location))
# df_location = pd.DataFrame({'location':house_location})

try:
    url[2].get_text().strip().replace('\n','').split()[0].strip()
except:
    int(url[2].get_text().strip().replace('\n','').split()[0].strip().replace(',',''))

SyntaxError: unexpected EOF while parsing (2587667026.py, line 7)

In [289]:
url

'https://streeteasy.com/building/halo-lic/2510'

In [288]:
url[2].get_text().strip().replace('\n','').split()[0].strip().replace(',','')

AttributeError: 'str' object has no attribute 'get_text'

In [183]:
location = num.find_all('p',{"class":"listingCardLabel listingCardLabel-grey listingCard-upperShortLabel"})
location

[<p class="listingCardLabel listingCardLabel-grey listingCard-upperShortLabel">
         Rental Unit in Crown Heights
       </p>]

In [180]:
num.find_all('p',{"class":"listingCardLabel listingCardLabel-grey listingCard-upperShortLabel"})

[<p class="listingCardLabel listingCardLabel-grey listingCard-upperShortLabel">
           New Development
         </p>,
 <p class="listingCardLabel listingCardLabel-grey listingCard-upperShortLabel">
         Rental Unit in Hunters Point
       </p>]

In [171]:
import re
test = re.match('(?<=in)*.',location.strip())

print(test)

<re.Match object; span=(0, 1), match='R'>


In [98]:
match = re.findall(r'(?<=in ).*', location.strip())
# Did we find a match?
print(match)

['Midtown']


'Rental Unit in Midtown'

## Some tests

In [1]:
# This is for all pages minium information 

# Some test
import pandas as pd
import itertools 
from bs4 import BeautifulSoup
import requests
from requests import get
import time
from random import seed
from random import random
from random import randint
from ediblepickle import checkpoint
import os
import sys
sys.setrecursionlimit(100000)

# url = first_page = 'https://streeteasy.com/for-rent/nyc'


    # initialize a list called houses 
houses = []
    # initialize variable count at 1

    # initialize variable new_count at 0

## make sure to get proxies with low latency and high success rate, even if you have to cut down the number of proxies used
proxies_list = [['https://149.56.106.104:3128'],
               ['https://35.245.208.185:3128']]

user_agent = [
        ['Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/42.0.2311.135 Safari/537.36 Edge/12.246'],
        ['Mozilla/5.0 (X11; CrOS x86_64 8172.45.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.64 Safari/537.36'],
        ['Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.111 Safari/537.36'],
        ['Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:15.0) Gecko/20100101 Firefox/15.0.1'],
        ['Mozilla/5.0 (Linux; U; Android 4.2.3; he-il; NEO-X5-116A Build/JDQ39) AppleWebKit/534.30 (KHTML, like Gecko) Version/5.0 Safari/534.30'],
        ['Mozilla/5.0 (X11; U; Linux armv7l like Android; en-us) AppleWebKit/531.2+ (KHTML, like Gecko) Version/5.0 Safari/533.2+ Kindle/3.0+'],
        ['Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.111 Safari/537.36'],
        ['Mozilla/5.0 (iPhone; CPU iPhone OS 12_0 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) CriOS/69.0.3497.105 Mobile/15E148 Safari/605.1']
        ]
# only one value per element
SecFetchUser = ['?1','?T','?F']
SecFetchSite = ['cross-site','same-origin','same-site']
SecFetchMode = ['websocket','no-cors','same-origin','cors']

## more than one value per element
AcceptEncoding = [['gzip','identity'],['gzip','deflate','identity'],['br','deflate','gzip'],
                 ['identity','br','deflate'],['gzip','deflate']]




AcceptLanguage = [['nl-NL,nl;q=0.'+str(randint(1,9)),'en-USA,en;q=0.'+str(randint(1,9))],
                  ['fr-CH,fr;q=0.'+str(randint(1,9)),'nl-NL,nl;q=0.'+str(randint(1,9))],
                  ['nl-NL,nl;q=0.'+str(randint(1,9)),'de;q=0.'+str(randint(1,9))],
                  ['nl-NL,nl;q=0.'+str(randint(1,9)),'de-CH;q=0.'+str(randint(1,9))]]



cache_dir = 'cache_streeteasy_v1'
if not os.path.exists(cache_dir):
    os.mkdir(cache_dir)

@checkpoint(key=lambda args, kwargs: args[0].replace('/','_') + '.pkl', work_dir=cache_dir)
def scrape_website(url):
    # initialize a list called houses 
    houses = []


    ## random variables
    value_fetchuser = randint(0,2)
    value_site = randint(0,2)
    value_mode = randint(0,3)
    value_userag = randint(0,7)

    str_user = ','.join(user_agent[value_userag])

    accptenco = randint(0,3)
    str3 = ','.join(AcceptEncoding[accptenco])
    accplang = randint(0,3)
    str4 = ','.join(AcceptLanguage[accplang])

    proxies_rand = ''.join(proxies_list[randint(0,1)])

    headers = {
        'Connection': 'keep-alive',
        'Cache-Control': 'max-age=0',
        'DNT': '0',
        'Upgrade-Insecure-Requests': '1',
        'User-Agent': str_user,
        'Sec-Fetch-User':SecFetchUser[value_fetchuser],
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.7,image/webp,image/apng,*/*;q=0.3,application/signed-exchange;v=b3',
        'Sec-Fetch-Dest':'none',
        'Sec-Fetch-Site': SecFetchSite[value_site],
        'Sec-Fetch-Mode': SecFetchMode[value_mode],
        'Accept-Encoding':str3,
        'Accept-Language': str4
    }

    # request the response
    response = get(url, headers=headers)

    # parse through the html 
    html_soup = BeautifulSoup(response.text, 'html.parser')
    # in the html of the page, find all the bins with <li> and class:
    #       house_data = html_soup.find_all('li', class_="search-list__item search-list__item--listing")
    house_data = html_soup.find_all('li', class_="searchCardList--listItem")
    # I like to print where the program is on the screen so we can follow its progress and where any errors happened
#    print(first_page)

    # if the response was not empty (if something was actually scraped)
    if house_data != []:
        # add to the list houses
        houses.extend(house_data)
        # random wait times
        value = randint(0,10)
        scaled_value = 15 + randint(-1,1)*randint(0,2)
        print(scaled_value)
        time.sleep(scaled_value)

    return houses


In [2]:
# Need to do a loop over all the pages

houses = []

# First webpage 
first_page = 'https://streeteasy.com/for-rent/nyc'

house_temp = scrape_website(first_page)

houses.extend(house_temp)

page_root = 'https://streeteasy.com/for-rent/nyc?page='

count = 1
count_2 = 0
while count < 1129:
    url = page_root + str(count)
    print(url)
    house = scrape_website(url)
    houses.extend(house)
    count += 1
    count_2 += 1;   
#     if count_2 == 20:
#         count_2 = 0
#         scaled_value = 15*60 + randint(-1,1)*randint(0,100) + randint(-1,1)*randint(0,100)
#         print(scaled_value)
#         time.sleep(scaled_value)
        



https://streeteasy.com/for-rent/nyc?page=1
https://streeteasy.com/for-rent/nyc?page=2
https://streeteasy.com/for-rent/nyc?page=3
https://streeteasy.com/for-rent/nyc?page=4
https://streeteasy.com/for-rent/nyc?page=5
https://streeteasy.com/for-rent/nyc?page=6
https://streeteasy.com/for-rent/nyc?page=7
https://streeteasy.com/for-rent/nyc?page=8
https://streeteasy.com/for-rent/nyc?page=9
https://streeteasy.com/for-rent/nyc?page=10
https://streeteasy.com/for-rent/nyc?page=11
https://streeteasy.com/for-rent/nyc?page=12
https://streeteasy.com/for-rent/nyc?page=13
https://streeteasy.com/for-rent/nyc?page=14
https://streeteasy.com/for-rent/nyc?page=15
https://streeteasy.com/for-rent/nyc?page=16
https://streeteasy.com/for-rent/nyc?page=17
https://streeteasy.com/for-rent/nyc?page=18
https://streeteasy.com/for-rent/nyc?page=19
https://streeteasy.com/for-rent/nyc?page=20
https://streeteasy.com/for-rent/nyc?page=21
https://streeteasy.com/for-rent/nyc?page=22
https://streeteasy.com/for-rent/nyc?page=

https://streeteasy.com/for-rent/nyc?page=188
https://streeteasy.com/for-rent/nyc?page=189
https://streeteasy.com/for-rent/nyc?page=190
https://streeteasy.com/for-rent/nyc?page=191
https://streeteasy.com/for-rent/nyc?page=192
https://streeteasy.com/for-rent/nyc?page=193
https://streeteasy.com/for-rent/nyc?page=194
https://streeteasy.com/for-rent/nyc?page=195
https://streeteasy.com/for-rent/nyc?page=196
https://streeteasy.com/for-rent/nyc?page=197
https://streeteasy.com/for-rent/nyc?page=198
https://streeteasy.com/for-rent/nyc?page=199
https://streeteasy.com/for-rent/nyc?page=200
https://streeteasy.com/for-rent/nyc?page=201
https://streeteasy.com/for-rent/nyc?page=202
https://streeteasy.com/for-rent/nyc?page=203
https://streeteasy.com/for-rent/nyc?page=204
https://streeteasy.com/for-rent/nyc?page=205
https://streeteasy.com/for-rent/nyc?page=206
https://streeteasy.com/for-rent/nyc?page=207
https://streeteasy.com/for-rent/nyc?page=208
https://streeteasy.com/for-rent/nyc?page=209
https://st

https://streeteasy.com/for-rent/nyc?page=378
https://streeteasy.com/for-rent/nyc?page=379
https://streeteasy.com/for-rent/nyc?page=380
https://streeteasy.com/for-rent/nyc?page=381
https://streeteasy.com/for-rent/nyc?page=382
https://streeteasy.com/for-rent/nyc?page=383
https://streeteasy.com/for-rent/nyc?page=384
https://streeteasy.com/for-rent/nyc?page=385
https://streeteasy.com/for-rent/nyc?page=386
https://streeteasy.com/for-rent/nyc?page=387
https://streeteasy.com/for-rent/nyc?page=388
https://streeteasy.com/for-rent/nyc?page=389
https://streeteasy.com/for-rent/nyc?page=390
https://streeteasy.com/for-rent/nyc?page=391
https://streeteasy.com/for-rent/nyc?page=392
https://streeteasy.com/for-rent/nyc?page=393
https://streeteasy.com/for-rent/nyc?page=394
https://streeteasy.com/for-rent/nyc?page=395
https://streeteasy.com/for-rent/nyc?page=396
https://streeteasy.com/for-rent/nyc?page=397
https://streeteasy.com/for-rent/nyc?page=398
https://streeteasy.com/for-rent/nyc?page=399
https://st

https://streeteasy.com/for-rent/nyc?page=569
https://streeteasy.com/for-rent/nyc?page=570
https://streeteasy.com/for-rent/nyc?page=571
https://streeteasy.com/for-rent/nyc?page=572
https://streeteasy.com/for-rent/nyc?page=573
https://streeteasy.com/for-rent/nyc?page=574
https://streeteasy.com/for-rent/nyc?page=575
https://streeteasy.com/for-rent/nyc?page=576
https://streeteasy.com/for-rent/nyc?page=577
https://streeteasy.com/for-rent/nyc?page=578
https://streeteasy.com/for-rent/nyc?page=579
https://streeteasy.com/for-rent/nyc?page=580
https://streeteasy.com/for-rent/nyc?page=581
https://streeteasy.com/for-rent/nyc?page=582
https://streeteasy.com/for-rent/nyc?page=583
https://streeteasy.com/for-rent/nyc?page=584
https://streeteasy.com/for-rent/nyc?page=585
https://streeteasy.com/for-rent/nyc?page=586
https://streeteasy.com/for-rent/nyc?page=587
https://streeteasy.com/for-rent/nyc?page=588
https://streeteasy.com/for-rent/nyc?page=589
https://streeteasy.com/for-rent/nyc?page=590
https://st

https://streeteasy.com/for-rent/nyc?page=758
https://streeteasy.com/for-rent/nyc?page=759
https://streeteasy.com/for-rent/nyc?page=760
https://streeteasy.com/for-rent/nyc?page=761
https://streeteasy.com/for-rent/nyc?page=762
https://streeteasy.com/for-rent/nyc?page=763
https://streeteasy.com/for-rent/nyc?page=764
https://streeteasy.com/for-rent/nyc?page=765
https://streeteasy.com/for-rent/nyc?page=766
https://streeteasy.com/for-rent/nyc?page=767
https://streeteasy.com/for-rent/nyc?page=768
https://streeteasy.com/for-rent/nyc?page=769
https://streeteasy.com/for-rent/nyc?page=770
https://streeteasy.com/for-rent/nyc?page=771
https://streeteasy.com/for-rent/nyc?page=772
https://streeteasy.com/for-rent/nyc?page=773
https://streeteasy.com/for-rent/nyc?page=774
https://streeteasy.com/for-rent/nyc?page=775
https://streeteasy.com/for-rent/nyc?page=776
https://streeteasy.com/for-rent/nyc?page=777
https://streeteasy.com/for-rent/nyc?page=778
https://streeteasy.com/for-rent/nyc?page=779
https://st

https://streeteasy.com/for-rent/nyc?page=950
https://streeteasy.com/for-rent/nyc?page=951
https://streeteasy.com/for-rent/nyc?page=952
https://streeteasy.com/for-rent/nyc?page=953
https://streeteasy.com/for-rent/nyc?page=954
https://streeteasy.com/for-rent/nyc?page=955
https://streeteasy.com/for-rent/nyc?page=956
https://streeteasy.com/for-rent/nyc?page=957
https://streeteasy.com/for-rent/nyc?page=958
https://streeteasy.com/for-rent/nyc?page=959
https://streeteasy.com/for-rent/nyc?page=960
https://streeteasy.com/for-rent/nyc?page=961
https://streeteasy.com/for-rent/nyc?page=962
https://streeteasy.com/for-rent/nyc?page=963
https://streeteasy.com/for-rent/nyc?page=964
https://streeteasy.com/for-rent/nyc?page=965
https://streeteasy.com/for-rent/nyc?page=966
https://streeteasy.com/for-rent/nyc?page=967
https://streeteasy.com/for-rent/nyc?page=968
https://streeteasy.com/for-rent/nyc?page=969
https://streeteasy.com/for-rent/nyc?page=970
https://streeteasy.com/for-rent/nyc?page=971
https://st

In [4]:
# This is for houses individually 

# Some test
import pandas as pd
import itertools 
from bs4 import BeautifulSoup
import requests
from requests import get
import time
from random import seed
from random import random
from random import randint
from ediblepickle import checkpoint
import os
import sys
sys.setrecursionlimit(100000)

# url = first_page = 'https://streeteasy.com/for-rent/nyc'


    # initialize a list called houses 
houses = []
    # initialize variable count at 1

    # initialize variable new_count at 0

## make sure to get proxies with low latency and high success rate, even if you have to cut down the number of proxies used
proxies_list = [['https://149.56.106.104:3128'],
               ['https://35.245.208.185:3128']]

user_agent = [
        ['Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/42.0.2311.135 Safari/537.36 Edge/12.246'],
        ['Mozilla/5.0 (X11; CrOS x86_64 8172.45.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.64 Safari/537.36'],
        ['Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.111 Safari/537.36'],
        ['Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:15.0) Gecko/20100101 Firefox/15.0.1'],
        ['Mozilla/5.0 (Linux; U; Android 4.2.3; he-il; NEO-X5-116A Build/JDQ39) AppleWebKit/534.30 (KHTML, like Gecko) Version/5.0 Safari/534.30'],
        ['Mozilla/5.0 (X11; U; Linux armv7l like Android; en-us) AppleWebKit/531.2+ (KHTML, like Gecko) Version/5.0 Safari/533.2+ Kindle/3.0+'],
        ['Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.111 Safari/537.36'],
        ['Mozilla/5.0 (iPhone; CPU iPhone OS 12_0 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) CriOS/69.0.3497.105 Mobile/15E148 Safari/605.1']
        ]
# only one value per element
SecFetchUser = ['?1','?T','?F']
SecFetchSite = ['cross-site','same-origin','same-site']
SecFetchMode = ['websocket','no-cors','same-origin','cors']

## more than one value per element
AcceptEncoding = [['gzip','identity'],['gzip','deflate','identity'],['br','deflate','gzip'],
                 ['identity','br','deflate'],['gzip','deflate']]




AcceptLanguage = [['nl-NL,nl;q=0.'+str(randint(1,9)),'en-USA,en;q=0.'+str(randint(1,9))],
                  ['fr-CH,fr;q=0.'+str(randint(1,9)),'nl-NL,nl;q=0.'+str(randint(1,9))],
                  ['nl-NL,nl;q=0.'+str(randint(1,9)),'de;q=0.'+str(randint(1,9))],
                  ['nl-NL,nl;q=0.'+str(randint(1,9)),'de-CH;q=0.'+str(randint(1,9))]]



cache_dir = 'cache_streeteasy_House_individual'
if not os.path.exists(cache_dir):
    os.mkdir(cache_dir)

@checkpoint(key=lambda args, kwargs: args[0].replace('/','_') + '.pkl', work_dir=cache_dir)
def scrape_website_individual_houses(url):
    # initialize a list called houses 
    houses = []


    ## random variables
    value_fetchuser = randint(0,2)
    value_site = randint(0,2)
    value_mode = randint(0,3)
    value_userag = randint(0,7)

    str_user = ','.join(user_agent[value_userag])

    accptenco = randint(0,3)
    str3 = ','.join(AcceptEncoding[accptenco])
    accplang = randint(0,3)
    str4 = ','.join(AcceptLanguage[accplang])

    proxies_rand = ''.join(proxies_list[randint(0,1)])

    headers = {
        'Connection': 'keep-alive',
        'Cache-Control': 'max-age=0',
        'DNT': '0',
        'Upgrade-Insecure-Requests': '1',
        'User-Agent': str_user,
        'Sec-Fetch-User':SecFetchUser[value_fetchuser],
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.7,image/webp,image/apng,*/*;q=0.3,application/signed-exchange;v=b3',
        'Sec-Fetch-Dest':'none',
        'Sec-Fetch-Site': SecFetchSite[value_site],
        'Sec-Fetch-Mode': SecFetchMode[value_mode],
        'Accept-Encoding':str3,
        'Accept-Language': str4
    }

    # request the response
    response = get(url, headers=headers)

    # parse through the html 
    html_soup = BeautifulSoup(response.text, 'html.parser')
    # in the html of the page, find all the bins with <li> and class:
    #       house_data = html_soup.find_all('li', class_="search-list__item search-list__item--listing")
#     house_data = html_soup.find_all('li', class_="searchCardList--listItem")
#     # I like to print where the program is on the screen so we can follow its progress and where any errors happened
# #    print(first_page)

#     # if the response was not empty (if something was actually scraped)
#     if house_data != []:
#         # add to the list houses
#         houses.extend(house_data)
#         # random wait times
    value = randint(0,10)
    scaled_value = 15 + randint(-1,1)*randint(0,2)
    print(scaled_value)
    time.sleep(scaled_value)

    return html_soup



In [6]:
count_2 = 0
for i in range(13381,len(result5.url)):
    url = result5.url[i]
    print(url)
    house = scrape_website_individual_houses(url)
    count_2 += 1;   
    if count_2 == 20:
        count_2 = 0
        scaled_value = 15*60 + randint(-1,1)*randint(0,100) + randint(-1,1)*randint(0,100)
        print(scaled_value)
        time.sleep(scaled_value)
        

https://streeteasy.com/building/70-pine-street-new_york/2407
17
https://streeteasy.com/building/55-riverwalk-place-west_new_york/355
15
https://streeteasy.com/building/425-5-avenue-new_york/52a
15
https://streeteasy.com/building/647-east-14-street-new_york/06d?featured=1
https://streeteasy.com/building/37-wall-street-new_york/10b?featured=1
https://streeteasy.com/building/55-riverwalk-place-west_new_york/355
https://streeteasy.com/building/425-5-avenue-new_york/52a
https://streeteasy.com/building/28_40-jackson-avenue-long_island_city/24j?infeed=1
https://streeteasy.com/building/50-park-plaza-drive-secaucus/222
15
https://streeteasy.com/building/8429-liberty-avenue-north_bergen/2
15
https://streeteasy.com/building/211-east-18-street-new_york/rental/3862741
16
https://streeteasy.com/building/kenton-place/16b
15
https://streeteasy.com/building/111-west-16-street-new_york/5
15
https://streeteasy.com/building/1381-myrtle-avenue-brooklyn/3f
15
https://streeteasy.com/building/10-charles-stree

15
https://streeteasy.com/building/99-atlantic-street-jersey_city/1
15
https://streeteasy.com/building/829-garfield-avenue-jersey_city/307
15
https://streeteasy.com/building/188-2-avenue-new_york/18
15
https://streeteasy.com/building/188-2-avenue-new_york/14
16
https://streeteasy.com/building/70-university-place-new_york/4?featured=1
https://streeteasy.com/building/151-east-80-street-new_york/04d?featured=1
14
https://streeteasy.com/building/40-brighton-1st-road-brooklyn/5k
13
https://streeteasy.com/building/23-avenue-b-manhattan/401
15
https://streeteasy.com/building/the-olivia/14f?infeed=1
17
https://streeteasy.com/building/260-avenue-east-bayonne/319
17
https://streeteasy.com/building/334-east-78-street-new_york/2
15
https://streeteasy.com/building/180-water-street-new_york/1702
13
https://streeteasy.com/building/200-west-86-street-new_york/5b
15
https://streeteasy.com/building/333-cypress-avenue-bronx-2/2c
17
https://streeteasy.com/building/150-greenway-terrace-forest_hills/28e
15


15
https://streeteasy.com/building/161-prince-street-new_york/15
17
https://streeteasy.com/building/waterside-plaza/1032k?featured=1
https://streeteasy.com/building/the-olivia/19f?featured=1
https://streeteasy.com/building/the-laureate/11f
17
https://streeteasy.com/building/the-chelsea-atelier/rental/3858974
15
https://streeteasy.com/building/365-bond/c316?infeed=1
910
https://streeteasy.com/building/77-west-washington-place-new_york/ph
15
https://streeteasy.com/building/lucent-condo/6d
16
https://streeteasy.com/building/93-worth/602
13
https://streeteasy.com/building/the-ashley/1209
15
https://streeteasy.com/building/the-ashley/1403
15
https://streeteasy.com/building/riverbend-at-port-imperial/0204
16
https://streeteasy.com/building/196-danforth-avenue-jersey_city/1b
15
https://streeteasy.com/building/1000-park-avenue-new_york/5b
15
https://streeteasy.com/building/the-parker-at-1894-lexington-avenue/7e
17
https://streeteasy.com/building/gotham-west/331?featured=1
https://streeteasy.co

15
https://streeteasy.com/building/856-8-avenue-new_york/3a?featured=1
14
https://streeteasy.com/building/the-chelsea/2d?featured=1
https://streeteasy.com/building/the-dylan/25b
15
https://streeteasy.com/building/105-clifton-place-jersey_city/2
16
https://streeteasy.com/building/eagle-west/20322?infeed=1
https://streeteasy.com/building/55-reade-st-new_york/4b
15
https://streeteasy.com/building/the-watermark/rental/3855887
15
https://streeteasy.com/building/131-imlay-street-brooklyn/3
16
https://streeteasy.com/building/67-wall-street-new_york/19j
17
https://streeteasy.com/building/hancock-manor/7a
15
https://streeteasy.com/building/407-east-81-street-new_york/4c
15
https://streeteasy.com/building/215-east-25-street-new_york/4a
14
https://streeteasy.com/building/88_10-34-avenue-jackson_heights/2e
15
https://streeteasy.com/building/225-east-21-street-new_york/ground-floor
15
https://streeteasy.com/building/waterside-plaza/4028h?featured=1
https://streeteasy.com/building/612-president-stre

15
https://streeteasy.com/building/425-5-avenue-new_york/56ab
15
https://streeteasy.com/building/1653-kennedy-boulevard-north_bergen/2
16
https://streeteasy.com/building/the-mason/18c
15
https://streeteasy.com/building/35-hudson-yards-manhattan/6701
15
https://streeteasy.com/building/103-east-86-street-new_york/2b
15
https://streeteasy.com/building/the-montana/8k
15
https://streeteasy.com/building/312-west-48-street-new_york/31
13
978
https://streeteasy.com/rental/3853568
15
https://streeteasy.com/building/104_60-queens-boulevard-queens/07l?featured=1
https://streeteasy.com/building/the-caledonia/408?featured=1
https://streeteasy.com/building/312-west-48-street-new_york/31
https://streeteasy.com/rental/3853568
https://streeteasy.com/building/linc-lic/2911?infeed=1
https://streeteasy.com/building/the-pier/1448
15
https://streeteasy.com/building/600-washington/202
15
https://streeteasy.com/building/353-east-83rd-street-new_york/16h
16
https://streeteasy.com/building/metropolitan-tower-co

...

16
https://streeteasy.com/building/marc-260-west-54-street-new_york/22h?featured=1
https://streeteasy.com/building/227-east-81-street-new_york/5w?featured=1
https://streeteasy.com/building/the-millennium-tower/rental/3809016
17
https://streeteasy.com/building/1005-walton-avenue-bronx/10e
15
https://streeteasy.com/building/the-greystone/214?infeed=1
https://streeteasy.com/building/magnolia-dumbo/231
16
https://streeteasy.com/building/149-grant-avenue-jersey_city/2a
13
https://streeteasy.com/building/the-buchanan/7n
17
https://streeteasy.com/building/1047-morris-avenue-bronx/4
14
https://streeteasy.com/building/1047-morris-avenue-bronx/1
17
https://streeteasy.com/building/22-mt-morris-park-west-new_york/7w
17
https://streeteasy.com/building/657-manida-street-bronx/3
16
https://streeteasy.com/building/8-east-83-street-new_york/pha
15
https://streeteasy.com/building/central-park-tower/rental/3807181
15
https://streeteasy.com/building/altalic-29_22-northern-boulevard-long_island_city/1709?f

15
https://streeteasy.com/building/the-greystone/214?infeed=1
https://streeteasy.com/building/425-grand-avenue-brooklyn/a6
15
https://streeteasy.com/building/avalon-cove/1307
15
https://streeteasy.com/building/715-east-5-street-new_york/2d
15
https://streeteasy.com/building/instrata-park-slope/11a
15
https://streeteasy.com/building/the-brooklyner/22e
15
https://streeteasy.com/building/177_18-wexford-terrace-jamaica/702
15
719
https://streeteasy.com/rental/3794116
15
https://streeteasy.com/building/442-east-20-street-new_york/03g
15
https://streeteasy.com/building/27-corbin-avenue-jersey_city/3f
16
https://streeteasy.com/building/37-wall-street-new_york/10b?featured=1
https://streeteasy.com/building/manhattan-park-10_40-river-road-new_york/02015j?featured=1
16
https://streeteasy.com/building/27-corbin-avenue-jersey_city/3f
https://streeteasy.com/building/15-west-29-street-new_york/2
17
https://streeteasy.com/building/the-greystone/214?infeed=1
https://streeteasy.com/rental/3793284
16
ht

14
https://streeteasy.com/building/jw-marriott-essex-house/1918
16
https://streeteasy.com/building/murray-hill-tower/20a
14
https://streeteasy.com/building/150-bleecker-street-brooklyn/4d
17
https://streeteasy.com/building/2020-hudson-street-fort_lee/505
15
https://streeteasy.com/building/beatrice-105-west-29th-street-new_york/31a?featured=1
https://streeteasy.com/building/the-addison-brooklyn/6c?featured=1
17
https://streeteasy.com/building/2020-hudson-street-fort_lee/505
https://streeteasy.com/building/2020-hudson-street-fort_lee/827
14
https://streeteasy.com/building/the-buchanan/p9?infeed=1
15
https://streeteasy.com/building/3013-barker-avenue-bronx/1d
17
https://streeteasy.com/building/247-west-63-street-new_york/phg
13
https://streeteasy.com/building/239-west-63-street-new_york/1g
15
https://streeteasy.com/building/18-avenue-a-new_york/3l
16
https://streeteasy.com/building/magnolia-dumbo/233
15
https://streeteasy.com/building/210-east-11-street-new_york/d
16
900
https://streeteas

15
https://streeteasy.com/building/24-wegman-parkway-jersey_city/2
15
https://streeteasy.com/building/518-east-83-street-new_york/rental/3789876
15
https://streeteasy.com/building/91-gorge-road-edgewater/1703
13
https://streeteasy.com/building/199-mott-street-new_york/ph1
15
https://streeteasy.com/building/37-wall-street-new_york/8h?featured=1
15
https://streeteasy.com/building/the-crossing-at-420-e-102nd-street/13f?featured=1
https://streeteasy.com/rental/3863509
15
https://streeteasy.com/rental/3863359
15
https://streeteasy.com/building/the-greystone/214?infeed=1
867
https://streeteasy.com/building/668-lenox-road-brooklyn/2
15
https://streeteasy.com/building/27e79-condominium/rental/3732407
15
https://streeteasy.com/building/425-west-205-street-new_york/5c
17
https://streeteasy.com/building/the-smile/1110
15
https://streeteasy.com/building/715-east-5-street-new_york/4d
15
https://streeteasy.com/building/the-sagamore/009n
16
https://streeteasy.com/building/atlantic-at-newport/1008
17


871
https://streeteasy.com/building/2464-hoffman-street-bronx/3
15
https://streeteasy.com/building/37-wall-street-new_york/10b?featured=1
https://streeteasy.com/building/79-warren-street-new_york/ph2?featured=1
15
https://streeteasy.com/building/651-west-42-street-new_york/rental/3548431
16
https://streeteasy.com/building/2010-hudson-street-fort_lee/601
15
https://streeteasy.com/building/the-greystone/214?infeed=1
https://streeteasy.com/building/5-main-street-edgewater/307
15
https://streeteasy.com/building/507-west-179-street-new_york/rental/3544221
16
https://streeteasy.com/rental/3543684
15
https://streeteasy.com/building/15-east-82-street-new_york/1
17
https://streeteasy.com/building/the-howell/1007
15
https://streeteasy.com/building/2020-hudson-street-fort_lee/1213
15
https://streeteasy.com/building/61_20-grand-central-parkway-forest_hills/c304
15
https://streeteasy.com/building/20_12-seagirt-boulevard-far_rockaway/06a
15
https://streeteasy.com/building/112-hoboken-avenue-jersey_c

In [5]:
result5[result5.url == 'https://streeteasy.com/building/1-2nd-st-1201-jc,_downtown/910']

,house_price,location,address,url,bedroom,bathroom,area
13380,3400,['NYC and NJ'],1 2nd Street #910,https://streeteasy.com/building/1-2nd-st-1201-...,1,1,765.0


In [34]:
scrape_website_individual_houses(result5.url[1140])

15



<!DOCTYPE html>

<html lang="en-US" xmlns="http://www.w3.org/1999/xhtml" xmlns:fb="http://www.facebook.com/2008/fbml" xmlns:og="http://ogp.me/ns#" xmlns:v="urn:schemas-microsoft-com:vml">
<head>
<link href="//cdn-assets-s3.streeteasy.com/assets/manifest-c93475b02bd2409b4a52e21af023e5d5f489f19500d234a3660fe4d35069bbac.json" rel="manifest"/>
<meta content="yes" name="apple-mobile-web-app-capable"/>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="ie=edge" http-equiv="x-ua-compatible"/>
<meta content="width=device-width, initial-scale=1.0, minimal-ui" name="viewport"/>
<title>444 Bergen Avenue #1L in Jersey City, New Jersey | StreetEasy</title>
<script src="https://polyfill.io/v3/polyfill.min.js?features=Intl"></script>
<script crossorigin="anonymous" integrity="sha384-Wh1kaX6+TxjSQdRRE+XUoIsEh5Cj8LgB6Wjbg/1/kTB3aG7q6ORaKSe5VfThKhcj" src="https://browser.sentry-cdn.com/6.3.6/bundle.min.js"></script>
<script>
    Sentry.init({
      dsn: "https://ba99ce3

In [754]:
    ## random variables
    value_fetchuser = randint(0,2)
    value_site = randint(0,2)
    value_mode = randint(0,3)
    value_userag = randint(0,7)

    str_user = ','.join(user_agent[value_userag])

    accptenco = randint(0,3)
    str3 = ','.join(AcceptEncoding[accptenco])
    accplang = randint(0,3)
    str4 = ','.join(AcceptLanguage[accplang])

    proxies_rand = ''.join(proxies_list[randint(0,1)])

    headers = {
        'Connection': 'keep-alive',
        'Cache-Control': 'max-age=0',
        'DNT': '0',
        'Upgrade-Insecure-Requests': '1',
        'User-Agent': str_user,
        'Sec-Fetch-User':SecFetchUser[value_fetchuser],
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.7,image/webp,image/apng,*/*;q=0.3,application/signed-exchange;v=b3',
        'Sec-Fetch-Dest':'none',
        'Sec-Fetch-Site': SecFetchSite[value_site],
        'Sec-Fetch-Mode': SecFetchMode[value_mode],
        'Accept-Encoding':str3,
        'Accept-Language': str4
    }

    # request the response
    response = get(result5.url[2], headers=headers)

In [756]:
response.text

'\n<!DOCTYPE html>\n<html xmlns="http://www.w3.org/1999/xhtml"\n      xmlns:v="urn:schemas-microsoft-com:vml"\n      xmlns:og="http://ogp.me/ns#"\n      xmlns:fb="http://www.facebook.com/2008/fbml"\n      lang="en-US">\n  <head>\n    <link rel="manifest" href="//cdn-assets-s3.streeteasy.com/assets/manifest-c93475b02bd2409b4a52e21af023e5d5f489f19500d234a3660fe4d35069bbac.json">\n\n    <meta name="apple-mobile-web-app-capable" content="yes" />\n    <meta http-equiv="Content-Type" content="text/html; charset=utf-8" />\n    <meta http-equiv="x-ua-compatible" content="ie=edge" />\n    <meta name="viewport" content="width=device-width, initial-scale=1.0, minimal-ui">\n\n      <title>19 Cypress Street GROUND-F in Jersey City, New Jersey | StreetEasy</title>\n\n\n\n\n    <script src="https://polyfill.io/v3/polyfill.min.js?features=Intl"></script>\n\n      <script\n    src="https://browser.sentry-cdn.com/6.3.6/bundle.min.js"\n    integrity="sha384-Wh1kaX6+TxjSQdRRE+XUoIsEh5Cj8LgB6Wjbg/1/kTB3aG7

In [30]:
result5.url[2000]

'https://streeteasy.com/building/31-avenue-c-new_york/22'

In [718]:
value = random()
scaled_value = 1 + (value * (9 - 5))

scaled_value

1.3564670767316627

In [582]:
value = random()
scaled_value = 6 + (value * (3))
scaled_value

6.18847214479945

In [601]:
len(houses)

1932

In [569]:
randint(-1,1)*randint(0,10) + randint(-1,1)*randint(0,10)

-11

In [562]:
value = random()
scaled_value = 1 + (value * (9 - 5))
print(scaled_value)


1.8526786954406846


In [619]:
# Some test
import pandas as pd
import itertools 
from bs4 import BeautifulSoup
import requests
from requests import get
import time
from random import seed
from random import random
from random import randint
from ediblepickle import checkpoint
import os
import sys
sys.setrecursionlimit(100000)

url = first_page = 'https://streeteasy.com/for-rent/nyc'


    # initialize a list called houses 
houses = []
    # initialize variable count at 1

    # initialize variable new_count at 0

## make sure to get proxies with low latency and high success rate, even if you have to cut down the number of proxies used
proxies_list = [['https://149.56.106.104:3128'],
               ['https://35.245.208.185:3128']]

user_agent = [
        ['Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/42.0.2311.135 Safari/537.36 Edge/12.246'],
        ['Mozilla/5.0 (X11; CrOS x86_64 8172.45.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.64 Safari/537.36'],
        ['Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.111 Safari/537.36'],
        ['Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:15.0) Gecko/20100101 Firefox/15.0.1'],
        ['Mozilla/5.0 (Linux; U; Android 4.2.3; he-il; NEO-X5-116A Build/JDQ39) AppleWebKit/534.30 (KHTML, like Gecko) Version/5.0 Safari/534.30'],
        ['Mozilla/5.0 (X11; U; Linux armv7l like Android; en-us) AppleWebKit/531.2+ (KHTML, like Gecko) Version/5.0 Safari/533.2+ Kindle/3.0+'],
        ['Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.111 Safari/537.36'],
        ['Mozilla/5.0 (iPhone; CPU iPhone OS 12_0 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) CriOS/69.0.3497.105 Mobile/15E148 Safari/605.1']
        ]
# only one value per element
SecFetchUser = ['?1','?T','?F']
SecFetchSite = ['cross-site','same-origin','same-site']
SecFetchMode = ['websocket','no-cors','same-origin','cors']

## more than one value per element
AcceptEncoding = [['gzip','identity'],['gzip','deflate','identity'],['br','deflate','gzip'],
                 ['identity','br','deflate'],['gzip','deflate']]




AcceptLanguage = [['nl-NL,nl;q=0.'+str(randint(1,9)),'en-USA,en;q=0.'+str(randint(1,9))],
                  ['fr-CH,fr;q=0.'+str(randint(1,9)),'nl-NL,nl;q=0.'+str(randint(1,9))],
                  ['nl-NL,nl;q=0.'+str(randint(1,9)),'de;q=0.'+str(randint(1,9))],
                  ['nl-NL,nl;q=0.'+str(randint(1,9)),'de-CH;q=0.'+str(randint(1,9))]]


# initialize a list called houses 
houses = []


## random variables
value_fetchuser = randint(0,2)
value_site = randint(0,2)
value_mode = randint(0,3)
value_userag = randint(0,7)

str_user = ','.join(user_agent[value_userag])

accptenco = randint(0,3)
str3 = ','.join(AcceptEncoding[accptenco])
accplang = randint(0,3)
str4 = ','.join(AcceptLanguage[accplang])

proxies_rand = ''.join(proxies_list[randint(0,1)])

headers = {
    'Connection': 'keep-alive',
    'Cache-Control': 'max-age=0',
    'DNT': '0',
    'Upgrade-Insecure-Requests': '1',
    'User-Agent': str_user,
    'Sec-Fetch-User':SecFetchUser[value_fetchuser],
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.7,image/webp,image/apng,*/*;q=0.3,application/signed-exchange;v=b3',
    'Sec-Fetch-Dest':'none',
    'Sec-Fetch-Site': SecFetchSite[value_site],
    'Sec-Fetch-Mode': SecFetchMode[value_mode],
    'Accept-Encoding':str3,
    'Accept-Language': str4
}

# request the response
response = get(url, headers=headers)

In [620]:
response

<Response [200]>

In [504]:
first_page = 'https://streeteasy.com/for-rent/nyc'

response = get(first_page)

In [527]:
headers = {"Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/22.0.1207.1 Safari/537.1"}

# request the response
with requests.Session() as s:
    response = s.get(url, headers=req_headers)



In [529]:
response

<Response [403]>

In [530]:
## make sure to get proxies with low latency and high success rate, even if you have to cut down the number of proxies used

user_agent = [
        ['Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/42.0.2311.135 Safari/537.36 Edge/12.246'],
        ['Mozilla/5.0 (X11; CrOS x86_64 8172.45.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.64 Safari/537.36'],
        ['Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.111 Safari/537.36'],
        ['Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:15.0) Gecko/20100101 Firefox/15.0.1'],
        ['Mozilla/5.0 (Linux; U; Android 4.2.3; he-il; NEO-X5-116A Build/JDQ39) AppleWebKit/534.30 (KHTML, like Gecko) Version/5.0 Safari/534.30'],
        ['Mozilla/5.0 (X11; U; Linux armv7l like Android; en-us) AppleWebKit/531.2+ (KHTML, like Gecko) Version/5.0 Safari/533.2+ Kindle/3.0+'],
        ['Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.111 Safari/537.36'],
        ['Mozilla/5.0 (iPhone; CPU iPhone OS 12_0 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) CriOS/69.0.3497.105 Mobile/15E148 Safari/605.1']
        ]
# only one value per element
SecFetchUser = ['?1','?T','?F']
SecFetchSite = ['cross-site','same-origin','same-site']
SecFetchMode = ['websocket','no-cors','same-origin','cors']

## more than one value per element
AcceptEncoding = [['gzip','identity'],['gzip','deflate','identity'],['br','deflate','gzip'],
                 ['identity','br','deflate'],['gzip','deflate']]




AcceptLanguage = [['nl-NL,nl;q=0.'+str(randint(1,9)),'en-USA,en;q=0.'+str(randint(1,9))],
                  ['fr-CH,fr;q=0.'+str(randint(1,9)),'nl-NL,nl;q=0.'+str(randint(1,9))],
                  ['nl-NL,nl;q=0.'+str(randint(1,9)),'de;q=0.'+str(randint(1,9))],
                  ['nl-NL,nl;q=0.'+str(randint(1,9)),'de-CH;q=0.'+str(randint(1,9))]]



# initialize a list called houses 
houses = []


## random variables
value_fetchuser = randint(0,2)
value_site = randint(0,2)
value_mode = randint(0,3)
value_userag = randint(0,7)

str_user = ','.join(user_agent[value_userag])

accptenco = randint(0,3)
str3 = ','.join(AcceptEncoding[accptenco])
accplang = randint(0,3)
str4 = ','.join(AcceptLanguage[accplang])

proxies_rand = ''.join(proxies_list[randint(0,1)])

headers = {
    'Connection': 'keep-alive',
    'Cache-Control': 'max-age=0',
    'DNT': '0',
    'Upgrade-Insecure-Requests': '1',
    'User-Agent': str_user,
    'Sec-Fetch-User':SecFetchUser[value_fetchuser],
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.7,image/webp,image/apng,*/*;q=0.3,application/signed-exchange;v=b3',
    'Sec-Fetch-Dest':'none',
    'Sec-Fetch-Site': SecFetchSite[value_site],
    'Sec-Fetch-Mode': SecFetchMode[value_mode],
    'Accept-Encoding':str3,
    'Accept-Language': str4
}

# request the response
response = get(url, headers=headers)

In [533]:
response

<Response [200]>

# Some test

In [637]:
from selenium import webdriver
path = '/Users/ftejera/Documents/GitRepositories/Capstone_Project_TDI/chromedriver'

driver = webdriver.Chrome(path)

driver.get('fttps://google.com')

/var/folders/pj/96hgn1jn0gz7jxtpbp8j5v5m0000gn/T/ipykernel_71585/98886539.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(path)


WebDriverException: Message: Can not connect to the Service /Users/ftejera/Documents/GitRepositories/Capstone_Project_TDI/chromedriver


In [648]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By

driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get("http://streeteasy.com")

searchbox = driver.find_element(By.XPATH,'//*[@id="aria-hidden-when-modal-open"]/div[2]/div[1]/div/div[2]/div/button')
searchbox.send_keys('Brooklyn')

searchButtom = driver.find_element(By.XPATH,'//*[@id="aria-hidden-when-modal-open"]/div[2]/div[1]/div/button')
searchButtom.click()


/var/folders/pj/96hgn1jn0gz7jxtpbp8j5v5m0000gn/T/ipykernel_71585/622259525.py:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="aria-hidden-when-modal-open"]/div[2]/div[1]/div/div[2]/div/button"}
  (Session info: chrome=103.0.5060.134)
Stacktrace:
0   chromedriver                        0x00000001053f5ef9 chromedriver + 4480761
1   chromedriver                        0x00000001053815d3 chromedriver + 4003283
2   chromedriver                        0x0000000105014338 chromedriver + 410424
3   chromedriver                        0x000000010504a5bd chromedriver + 632253
4   chromedriver                        0x000000010504a841 chromedriver + 632897
5   chromedriver                        0x000000010507c7d4 chromedriver + 837588
6   chromedriver                        0x0000000105067a8d chromedriver + 752269
7   chromedriver                        0x000000010507a4f1 chromedriver + 828657
8   chromedriver                        0x0000000105067953 chromedriver + 751955
9   chromedriver                  

In [645]:
from selenium import webdriver

driver = webdriver.Chrome()

WebDriverException: Message: 'chromedriver' executable needs to be in PATH. Please see https://chromedriver.chromium.org/home


In [654]:
import requests

url = "https://zillow56.p.rapidapi.com/search"

querystring = {"location":"houston, tx"}

headers = {
	"X-RapidAPI-Key": "31462bea3fmshdfc389233c30a35p10fb0bjsnbab486ac6855",
	"X-RapidAPI-Host": "zillow56.p.rapidapi.com"
}

response = requests.request("GET", url, headers=headers, params=querystring)

print(response.text)

{"message":"You are not subscribed to this API."}
